# Import

In [9]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import sqlite3

# Read In

In [10]:
referrals_big_geo = pd.read_csv('../data/neo4j/referrals_big_geo.csv')

C:\Users\jrior\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
from_communities = pd.read_csv('../data/communities/from_communities.csv', usecols = ['from_npi', 'from_communityid'])

In [12]:
to_communities = pd.read_csv('../data/communities/to_communities.csv', usecols = ['to_npi', 'to_communityid'])

# Create SQLite database: hopteam_broad

In [18]:
db = sqlite3.connect('../data/hopteam_broad.sqlite')    

with sqlite3.connect('../data/hopteam_broad.sqlite') as db:
    referrals_big_geo.to_sql('referrals_geo', db, if_exists = 'append', index = False)

In [19]:
with sqlite3.connect('../data/hopteam_broad.sqlite') as db:
    from_communities.to_sql('communities_from', db, if_exists = 'append', index = False)

In [20]:
with sqlite3.connect('../data/hopteam_broad.sqlite') as db:
    to_communities.to_sql('communities_to', db, if_exists = 'append', index = False)

# Test out hopteam_broad

In [25]:
%%time
# Get all rows of referrals_geo together with communities from Louvain algorithm
query = """
     
     SELECT *
     FROM referrals_geo
     LEFT JOIN communities_from
         USING (from_npi)
     LEFT JOIN communities_to
         USING (to_npi)

"""

with sqlite3.connect('../data/hopteam_broad.sqlite') as db:
    all_fields = pd.read_sql(query, db)
    
all_fields

Wall time: 5.3 s


,from_npi,referral_id,entity_from,name_from,organization_from,taxonomy_code_from,classification_from,specialization_from,address_line1_from,address_line2_from,...,std_day_wait,address_from,address_to,lat_from,lng_from,lat_to,lng_to,distance_miles,from_communityid,to_communityid
0,1821080961,18854880,1.0,"SHERWOOD, WILLIAM",None,207Q00000X,Family Medicine,None,302 N CONGRESS BLVD,None,...,40.266,"302 N CONGRESS BLVD, SMITHVILLE, TN 37166","400 E PUBLIC SQUARE, ALEXANDRIA, TN 37012",35.969171,-85.807472,36.077559,-86.033322,14.665166,20624.0,20624.0
1,1679677199,104890727,2.0,None,"CRIPPS, HOOPER & RHODY, PLLC",207Q00000X,Family Medicine,None,302 N CONGRESS BLVD,None,...,0.000,"302 N CONGRESS BLVD, SMITHVILLE, TN 37166","400 E PUBLIC SQUARE, ALEXANDRIA, TN 37012",35.969171,-85.807472,36.077559,-86.033322,14.665166,NaN,20624.0
2,1841282779,131966153,1.0,"RHODY, KEVIN",None,207Q00000X,Family Medicine,None,302 N CONGRESS BLVD,None,...,36.447,"302 N CONGRESS BLVD, SMITHVILLE, TN 37166","400 E PUBLIC SQUARE, ALEXANDRIA, TN 37012",35.969171,-85.807472,36.077559,-86.033322,14.665166,20624.0,20624.0
3,1770575607,146169993,1.0,"COOPER, STEVEN",None,207Q00000X,Family Medicine,None,302 N CONGRESS BLVD,None,...,18.756,"302 N CONGRESS BLVD, SMITHVILLE, TN 37166","400 E PUBLIC SQUARE, ALEXANDRIA, TN 37012",35.969171,-85.807472,36.077559,-86.033322,14.665166,20624.0,20624.0
4,1093741464,29450644,2.0,None,"ADVANCED DIAGNOSTIC IMAGING, PC",207R00000X,Internal Medicine,None,3024 BUSINESS PARK CIR,None,...,43.394,"3024 BUSINESS PARK CIR, GOODLETTSVILLE, TN 37072","400 E PUBLIC SQUARE, ALEXANDRIA, TN 37012",36.332108,-86.700830,36.077559,-86.033322,41.132856,NaN,20624.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240562,1245257005,191055279,2.0,None,CARDIOLOGY AND INTERNAL MEDICINE GROUP OF NORT...,207R00000X,Internal Medicine,None,155 AVENUE E,None,...,9.270,"155 AVENUE E, APALACHICOLA, FL 32320","1510 HATCHER LN, SUITE 1, COLUMBIA, TN 38401",29.721553,-84.991627,35.603333,-87.061742,423.526420,NaN,NaN
240563,1245623701,191233185,2.0,None,L & K HEALTH SERVICES,363LF0000X,Nurse Practitioner,Family,1004 HICKORY HILL LN,SUITE 106,...,0.000,"1004 HICKORY HILL LN, SUITE 106, HERMITAGE, TN...","1324 TROTWOOD AVE STE 1, COLUMBIA, TN 38401",36.189579,-86.629045,35.600876,-87.072753,47.624578,NaN,NaN
240564,1205875713,184518074,2.0,None,ACCELERATED REHABILITATION CENTERS LTD,225100000X,Physical Therapist,None,600 OAKMONT LN STE 600C,None,...,0.000,"600 OAKMONT LN STE 600C, WESTMONT, IL 60559","1810 SHADY BROOK ST STE 4, COLUMBIA, TN 38401",41.821708,-87.960959,35.602084,-87.057549,432.176893,NaN,NaN
240565,1558441311,31085666,2.0,None,KAU HOSPITAL,282NC0060X,General Acute Care Hospital,Critical Access,1 KAMANI STREET,None,...,78.282,"1 KAMANI STREET, PAHALA, HI 96777","166 FIFTH RD, SUMMERTOWN, TN 38483",19.199622,-155.472070,35.484525,-87.341322,4245.485254,NaN,NaN
